In [2]:
import numpy as np
import pandas as pd
import random
import sys
from sklearn.preprocessing import OneHotEncoder
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import LabelPropagation
from scipy import sparse
from scipy.sparse import csgraph
from sklearn.preprocessing import minmax_scale

## City data

In [21]:
dataset = pd.read_csv("predict_e_data.csv")

In [34]:
city_data = dataset.groupby(["city","years", "companies"]).mean().reset_index().loc[:, ["city", "years", "companies", "predict_label"]]

In [35]:
city_data.head()

,city,years,companies,predict_label
0,'S GRAVENHAGE,2015,stedin,1.666667
1,'S GRAVENHAGE,2016,stedin,2.375000
2,'S GRAVENHAGE,2017,stedin,2.333333
3,'S GRAVENHAGE,2019,stedin,3.000000
4,'S GRAVENMOER,2010,enexis,2.560976


In [36]:
city_data["predict_label"] = city_data["predict_label"].apply(lambda x: int(x))

In [38]:
city_data.to_csv("e_city_data.csv", index = False)

In [39]:
dataset = pd.read_csv("predict_g_data.csv")
city_data = dataset.groupby(["city","years", "companies"]).mean().reset_index().loc[:, ["city", "years", "companies", "predict_label"]]
city_data["predict_label"] = city_data["predict_label"].apply(lambda x: int(x))
city_data.to_csv("g_city_data.csv", index = False)

## Low-T and smartmeter

In [3]:
dataset = pd.read_csv("predict_e_data.csv")

In [9]:
L_and_S = dataset.groupby(["city","years", "companies"]).mean().reset_index().loc[:, ["city", "years", "companies", "smartmeter_perc", "annual_consume_lowtarif_perc"]]

In [13]:
annual_consume = dataset.groupby(["city","years", "companies"]).sum().reset_index().loc[:, ["city", "years", "companies", "annual_consume"]]

In [18]:
L_S_and_consume = pd.merge(left = L_and_S, right = annual_consume, left_on = ["city","years", "companies"], right_on = ["city","years", "companies"])

In [22]:
L_S_and_consume.to_csv("mean_LS.csv", index = False, float_format = "%g")

In [3]:
L_S_and_consume = pd.read_csv("mean_LS.csv")

In [5]:
L_S_and_consume["annual_consume"] = L_S_and_consume["annual_consume"] / 1000

In [8]:
L_S_and_consume.to_csv("mean_LS.csv", index = False, float_format = "%g")

In [11]:
np.max(L_S_and_consume["annual_consume"])

55832.6

In [12]:
np.min(L_S_and_consume["annual_consume"])

1.249

In [15]:
minmax_scale(L_S_and_consume["annual_consume"],feature_range=(2, 20))

array([2.00328299, 2.01068303, 2.00365246, ..., 2.00110454, 2.00108262,
       2.00109261])

## Geocode

In [2]:
e_geocode = pd.read_csv("eee.csv")

In [6]:
e_geocode.loc[e_geocode["years"] == 2010]

,city,years,companies,predict_label,latitude,longtitude
0,S-GRAVENHAGE,2010,liander,3,52.083333,4.300000
1,S-GRAVENHAGE,2010,stedin,2,52.083333,4.300000
29,ROTTERDAM,2010,liander,2,51.916667,4.500000
30,ROTTERDAM,2010,stedin,2,51.916667,4.500000
49,UTRECHT,2010,liander,1,52.093813,5.119095
50,UTRECHT,2010,stedin,2,52.093813,5.119095
66,EINDHOVEN,2010,enexis,2,51.450000,5.466667
76,HAARLEMMERLIEDE,2010,liander,2,52.388873,4.687978
86,GRONINGEN,2010,enexis,2,53.216667,6.550000
96,ARNHEM,2010,liander,2,51.979818,5.925636


## Electricity Consume file

In [162]:
data = pd.read_csv("mean_LS.csv")

In [163]:
company_data = data.loc[data.groupby(["city", "years"])["annual_consume"].idxmax(), ["city", "years", "companies"]].reset_index(drop = True)

In [164]:
consume_data = data.groupby(["city", "years"])["annual_consume"].sum().reset_index()

In [165]:
consume_data["annual_consume"] = consume_data["annual_consume"].apply(lambda x: round(x,3))

In [166]:
page2_e_consume_data = company_data.merge(right = consume_data, how = "left", on = ["city", "years"])

In [167]:
page2_e_consume_data.to_csv("e_consume_data.csv", index = False, float_format = "%g")

## Electricity Lowtarif and smartmeter file

In [45]:
companies = ["enexis", "liander", "stedin"]
features = ["city", "annual_consume_lowtarif_perc", "smartmeter_perc"]
Range = list(range(0,100,10))
Range.append(101)
labels = [5,15,25,35,45,55,65,75,85,95]
data = pd.DataFrame()
for year in range(2010, 2020):
    for company in companies:
        raw_data = pd.read_csv("electricity/%s_electricity_%s.csv" % (company, year))[features].dropna()
        raw_data["years"] = year
        raw_data["lowtarif_cut"] = pd.cut(raw_data["annual_consume_lowtarif_perc"].values, Range, right = False, labels = labels)
        raw_data["smartmeter_cut"] = pd.cut(raw_data["smartmeter_perc"].values, Range, right = False, labels = labels)
        raw_data = raw_data.iloc[:,[0,3,4,5]]
        data = data.append(raw_data, ignore_index = True)


In [46]:
l_data = data.groupby(["city", "years", "lowtarif_cut"]).count().reset_index()
l_data = l_data.rename(columns = {"smartmeter_cut":"lowtarif_cut_count"}).dropna().reset_index(drop = True)
l_data["lowtarif_cut_count"] = l_data["lowtarif_cut_count"].apply(lambda x: int(x))
l_data.to_csv("e_lowtarif_count.csv", index = False)
l_data = pd.read_csv("e_lowtarif_count.csv")
l_data = l_data.merge(right = company_data, how = "left", on = ["city", "years"])
l_data.to_csv("e_lowtarif_count.csv", index = False)

s_data = data.groupby(["city", "years", "smartmeter_cut"]).count().reset_index()
s_data = s_data.rename(columns = {"lowtarif_cut":"smartmeter_cut_count"}).dropna().reset_index(drop = True)
s_data["smartmeter_cut_count"] = s_data["smartmeter_cut_count"].apply(lambda x: int(x))
s_data.to_csv("e_smartmeter_count.csv", index = False)
s_data = pd.read_csv("e_smartmeter_count.csv")
s_data = s_data.merge(right = company_data, how = "left", on = ["city", "years"])
s_data.to_csv("e_smartmeter_count.csv", index = False)

## Gas consume


In [152]:
companies = ["enexis", "liander", "stedin"]
features = ["city", "annual_consume"]
data = pd.DataFrame()
for year in range(2010,2020):
    for company in companies:
        raw_data = pd.read_csv("gas/%s_gas_%s.csv" % (company, year))[features].dropna()
        raw_data["years"] = year
        raw_data["companies"] = company
        data = data.append(raw_data, ignore_index = True)

In [153]:
consume_data = data.groupby(["city", "years", "companies"]).sum().reset_index()

In [154]:
company_data = consume_data.loc[consume_data.groupby(["city", "years"])["annual_consume"].idxmax(), ["city", "years", "companies"]].reset_index(drop = True)

In [155]:
consume_data = consume_data.groupby(["city", "years"])["annual_consume"].sum().reset_index()

In [156]:
consume_data["annual_consume"] = (consume_data["annual_consume"] / 1000).apply(lambda x: round(x, 3))

In [157]:
page2_g_consume_data = company_data.merge(right = consume_data, how = "left", on = ["city", "years"])

In [158]:
page2_g_consume_data.to_csv("g_consume_data.csv", index = False, float_format = "%g")

In [172]:
np.min(page2_g_consume_data)

city              'S GRAVENDEEL
years                      2010
companies                enexis
annual_consume            0.818
dtype: object

## Gas annual consume per conn and smartmeter

In [132]:
companies = ["enexis", "liander", "stedin"]
features = ["city", "num_connections",  "annual_consume", "smartmeter_perc"]
Range = list(range(0,100,10))
Range.append(101)
labels = [5,15,25,35,45,55,65,75,85,95]
consume_range = list(range(0, 200, 20))
consume_range.append(2600)
c_labels = [10,30,50,70,90,110,130,150,170,190]


data = pd.DataFrame()
for year in range(2010, 2020):
    for company in companies:
        raw_data = pd.read_csv("gas/%s_gas_%s.csv" % (company, year))[features].dropna()
        raw_data["years"] = year
        data = data.append(raw_data, ignore_index = True)
data["c_per_conn"] = data["annual_consume"] / data["num_connections"]
data["c_per_conn_cut"] = pd.cut(data["c_per_conn"].values, consume_range, right = False, labels = c_labels)
data["smartmeter_cut"] = pd.cut(data["smartmeter_perc"].values, Range, right = False, labels = labels)
data = data.iloc[:,[0,4,6,7]]

In [133]:
c_data = data.groupby(["city", "years", "c_per_conn_cut"]).count().reset_index()
c_data = c_data.rename(columns = {"smartmeter_cut":"c_per_conn_cut_count"}).dropna().reset_index(drop = True)
c_data["c_per_conn_cut_count"] = c_data["c_per_conn_cut_count"].apply(lambda x: int(x))
c_data.to_csv("g_c_per_conn_count.csv", index = False)
c_data = pd.read_csv("g_c_per_conn_count.csv")
c_data = c_data.merge(right = company_data, how = "left", on = ["city", "years"])
c_data.to_csv("g_c_per_conn_count.csv", index = False)

s_data = data.groupby(["city", "years", "smartmeter_cut"]).count().reset_index()
s_data = s_data.rename(columns = {"c_per_conn_cut":"smartmeter_cut_count"}).dropna().reset_index(drop = True)
s_data["smartmeter_cut_count"] = s_data["smartmeter_cut_count"].apply(lambda x: int(x))
s_data.to_csv("g_smartmeter_count.csv", index = False)
s_data = pd.read_csv("g_smartmeter_count.csv")
s_data = s_data.merge(right = company_data, how = "left", on = ["city", "years"])
s_data.to_csv("g_smartmeter_count.csv", index = False)

## Gas mean c_per_conn and smartmeter

In [97]:
companies = ["enexis", "liander", "stedin"]
features = ["city", "num_connections",  "annual_consume", "smartmeter_perc"]
data = pd.DataFrame()
for year in range(2010, 2020):
    for company in companies:
        raw_data = pd.read_csv("gas/%s_gas_%s.csv" % (company, year))[features].dropna()
        raw_data["years"] = year
        data = data.append(raw_data, ignore_index = True)

In [98]:
data["c_per_conn"] = data["annual_consume"] / data["num_connections"]

In [107]:
data = data.iloc[:,[0,2,3,4,5]].groupby(["city", "years"]).agg({"annual_consume": "sum", "smartmeter_perc": "mean", "c_per_conn": "mean"}).reset_index()

In [108]:
mean_CS = company_data.merge(right = data, how = "left", on = ["city", "years"])

In [110]:
mean_CS.to_csv("mean_CS.csv", index = False, float_format = "%g")

In [113]:
mean_CS = pd.read_csv("mean_CS.csv")
data = pd.DataFrame(mean_CS.loc[mean_CS["years"] == int(year), ["smartmeter_perc", "c_per_conn", "annual_consume", "city", "companies"]]).reset_index(drop = True)
data["annual_consume"] = minmax_scale(data["annual_consume"],feature_range=(2, 20))

In [116]:
np.max(mean_CS["c_per_conn"])

913.7710000000001